In [1]:
import process as pp
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import CNN_LSTM
from CNN_LSTM import CNN_LSTM  # Add this line to import the class specifically
from imblearn.over_sampling import SMOTE
from CNN_LSTM_2 import CNN_LSTM_2 # 
from collections import Counter

class SensorDataset(Dataset):
    def __init__(self, windows, labels):
        self.windows = windows
        self.labels = labels
        
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        window = torch.tensor(self.windows[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return window, label


# Hyperparameters

num_channels = 6
batch_size = 32
num_epochs = 1000
lr = 0.0001

fs = 60
window_duration_sec = 1
overlap = 0.5

# Load and pre-process the data.
X, y = pp.load_and_preprocess_data('./Train_data', window_duration_sec, fs, overlap)
y_encoded, label_to_idx = pp.encode_labels(y)
print("Label to index mapping:", label_to_idx)
print("X shape:", X.shape)  
print("y shape:", y_encoded.shape)

unique, counts = np.unique(y_encoded, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Number of data points per class:", class_counts)


n_samples, window_length, num_channels = X.shape
#X_flat = X.reshape(n_samples, window_length * num_channels)
#print("Flattened X shape:", X_flat.shape)  
#Smote, not sure if it works now when we have timne series data
#smote = SMOTE(random_state=198)

#X_res_flat, y_res = smote.fit_resample(X_flat, y)
#print("New label distribution:", Counter(y_res))
#X_res = X_res_flat.reshape(-1, window_length, num_channels)
#print("Resampled X shape:", X_res.shape)

# Split into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=19, stratify=y_encoded)

window_length = fs * window_duration_sec

#ok to shuffel as each window should be fine to be shuffled
train_dataset = SensorDataset(X_train, y_train)
test_dataset = SensorDataset(X_test, y_test)

# Wrap the datasets in DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



Loading ./Train_data/Jacob_running.csv
The average frequency is approximately 59.08 Hz
Loaded 5820 samples
Applied low-pass filter
Segmented into 193 windows
Loading ./Train_data/Jacob_first_gym.csv
The average frequency is approximately 59.65 Hz
Loaded 7105 samples
Applied low-pass filter
Segmented into 235 windows
Loading ./Train_data/Julia_first_gym.csv
The average frequency is approximately 61.00 Hz
Loaded 5993 samples
Applied low-pass filter
Segmented into 198 windows
Loading ./Train_data/Julia_running.csv
The average frequency is approximately 54.71 Hz
Loaded 4971 samples
Applied low-pass filter
Segmented into 164 windows
Loading ./Train_data/julia_sitting_to_fall.csv
The average frequency is approximately 60.64 Hz
Loaded 4987 samples
Applied low-pass filter
Segmented into 165 windows
Loading ./Train_data/Marten_second.csv
The average frequency is approximately 60.08 Hz
Loaded 4800 samples
Applied low-pass filter
Segmented into 159 windows
Loading ./Train_data/Marten_first.csv
Th

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = CNN_LSTM_2(
    num_channels=num_channels, 
    num_classes=len(label_to_idx),  # or 'num_classes' if you prefer
    window_length=window_length
)  

# Train the model
model = pp.train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        device=device,
        num_epochs=num_epochs,
        lr=lr
    )
torch.save(model.state_dict(), "cnn_lstm_model_2.pth")
print("Model saved to cnn_lstm_model_2.pth")

Using device: cpu
Epoch [1/1000] Train Loss: 1.8982, Train Acc: 28.49% | Test Loss: 1.7285, Test Acc: 34.76%
Epoch [2/1000] Train Loss: 1.6902, Train Acc: 34.86% | Test Loss: 1.6810, Test Acc: 34.76%
Epoch [3/1000] Train Loss: 1.6807, Train Acc: 34.86% | Test Loss: 1.6798, Test Acc: 34.76%
Epoch [4/1000] Train Loss: 1.6799, Train Acc: 34.86% | Test Loss: 1.6796, Test Acc: 34.76%
Epoch [5/1000] Train Loss: 1.6806, Train Acc: 34.86% | Test Loss: 1.6794, Test Acc: 34.76%
Epoch [6/1000] Train Loss: 1.6804, Train Acc: 34.86% | Test Loss: 1.6794, Test Acc: 34.76%
Epoch [7/1000] Train Loss: 1.6800, Train Acc: 34.86% | Test Loss: 1.6794, Test Acc: 34.76%
Epoch [8/1000] Train Loss: 1.6784, Train Acc: 34.86% | Test Loss: 1.6717, Test Acc: 34.76%
Epoch [9/1000] Train Loss: 1.5940, Train Acc: 34.86% | Test Loss: 1.4602, Test Acc: 34.76%
Epoch [10/1000] Train Loss: 1.6120, Train Acc: 34.86% | Test Loss: 1.5612, Test Acc: 34.76%
Epoch [11/1000] Train Loss: 1.4594, Train Acc: 34.86% | Test Loss: 1.51

KeyboardInterrupt: 